In [ ]:
%pylab

# Part III: Evaluating Impact of Uncertainties on Compton Imaging

In the previous lesson, we succeeded in applying a simple Compton backprojection algorithm to create Compton images. If you have ever seen a Compton image before, you were probably surprised at how well the point source was recovered using only a simple backprojection algorithm. This is because we have been using *perfect* simulated data; i.e. we had ignored how the *detector response* impacts the collected data, and ultimately the imaging.

In this lesson, we will begin to explore how various aspects of detector response impact Compton imaging. In particular, we will focus on the impact of detector energy and position resolution, as these are the primary factors affecting image resolution. 

It is important to keep in mind that these exercises are very superficial and designed only to provide a tangible example of how real-world issues affect gamma-ray imaging systems. In reality, detector responses are far more complex than the simple models we will use, and there are many other factors that are neither captured nor even considered here. That being said, simple analyses like these are often valuable in providing basic insight and order-of-magnitude estimates for effects that you may encounter in true imaging applications.

## Getting Started

We begin by summarizing the progress we've made in the last two lessons: loading the data, selecting a subset of imagible photopeak events, and computing the relevant Compton data.

In [ ]:
# Load the data
import tables
with tables.open_file("hits.h5", "r") as hf:
    event_pointers = hf.root.EventPointers.read()
    event_lengths = hf.root.EventLengths.read()
    idata = hf.root.InteractionData.read()

In [ ]:
# Create an array of double-interaction events
num_interactions_per_event = 2
evmask = event_lengths == num_interactions_per_event
ptrs = event_pointers[evmask]
doubles = np.array([idata[p:p+num_interactions_per_event] for p in ptrs])

In [ ]:
# Extract full-energy events
ens = doubles['energy'].sum(axis=1)   # Total energy deposited in both interactions
ppk_mask = (ens >= 661.) & (ens <= 662.)
# Select the photopeak events
ppk_evs = doubles[ppk_mask]

If you had enough time to wrap your Compton cone computations up into functions, you can import them and apply them to the data below. If not, we've provided our version of those functions in the `event_processing.py` module.

In [ ]:
from event_processing import compute_cone_opening_angle, compute_cone_scatter_axes

We will create Compton data for the "perfect" events with no detector response to be used as a basis for comparing our detector response models against.

In [ ]:
E = 661.657 # keV
num_cones = 1000   # Use the first N cones for imaging
perfect_cones = (compute_cone_scatter_axes(ppk_evs)[:num_cones,:], compute_cone_opening_angle(E, ppk_evs['energy'][:,0])[:num_cones])

Instantiate a backprojector to compute our images.

In [ ]:
from compton_imaging import ComptonBackprojection2D
backprojector = ComptonBackprojection2D()

In [ ]:
# Perfect image
img = backprojector.backproject_cones(*perfect_cones)

In [ ]:
plt.imshow(img, extent=backprojector.extent, interpolation="none");

## Energy Resolution

It is clear from the Compton kinematic equations that uncertainty in the deposited and total energies of the gamma-ray will contribute to an uncertainty in the opening angle of the Compton cone. This of course has an impact on the overal image resolution.

We can probe this effect by applying various uncertainty models to the energy information in the simulated data. If we assume that we know the incident energy of the photon very precisely from isotopic identification (N.B. this is one of the advantages of using spectroscopic information to define the incident photon energy), then the only term we need modify is the energy desposited in the Compton scattering interaction.

### Exercise

Assume that the detector has 0.5% energy resolution over the range from 0 keV to 3 MeV (this is a VERY simplified model, but 0.5% energy resolution is of the right order for an HPGe strip detector). Apply this uncertainty model to the simulated data.

Create an energy spectrum for the "perfect" imaging events, and the events with this energy uncertainty. Visually confirm that the energy spectrum reflects your applied uncertainty model.

Create a Compton image using the blurred energies and compare it to the "perfect" image.

Note that plotting the images side-by-side may not capture the differences between the images (especially if the color-scales are different). Consider taking a slice through the maximum of the image or projecting it to 1D to compare the differences.

#### Extension

If you find this interesting and have the time, consider repeating the above analysis for different energy uncertainty models. For example, you could apply the same model with different values for the energy uncertainty (say 1.5% for CZT, 3-5% for a good spectroscopic scintillator (e.g. LaBr), and 8-10% for NaI).

You could also try using a more realistic energy uncertainty model. For example, rather than assuming the energy resolution of the detector is constant over the energy range, a higher uncertainty could be used for lower-energy gamma-rays to account for electronic noise. 

## Position Resolution

Another factor that has a large impact on the image quality (resolution, SNR, etc.) of Compton images is uncertainty in the position of gamma-ray interactions within the detector. Uncertainty in the interaction positions manifest as an uncertainty in the scattering axis of the Compton cone. The relationship between position uncertainty and angular resolution is more complicated than the relationship with energy resolution as the uncertainty in the cone scattering axis also depends on the distance between the first two interaction locations, often referred to as the *lever-arm*.

In this section, we explore the affects of position resolution on image quality in much the same way as above: applying uncertainty models to the position data and qualitatively comparing the output images.

### Exercise

The detector that was modelled for the simulation has a double-sided orthogonal strip readout with a strip pitch of 2mm. It has been [claimed in the literature](https://www.sciencedirect.com/science/article/pii/S0168900206017694) that this system can achieve roughly $400 \mu m$ FWHM position resolution. This gives us a decent idea for a simple model of position uncertainty. Let's assume that the position resolution has the same magnitude in all three dimensions, and that the distribution can be modelled by a simple Gaussian.

Apply the uncertainty model to the simulated data, and compare the resulting image.

## Event Selection

If everything went well with the above exercise, you will have noticed a much more significant blurring affect from the position resolution than was seen for the energy resolution component. However, as was mentioned above, the relationship between position uncertainty and image resolution also depends on the *lever-arm* for the event. This offers us a straight-forward way to control the image quality by eliminating events with short lever-arms. This process is a form of *event selection*, and is an example of one of the many ways that the tradeoff between *resolution* and *efficiency* can be explored in Compton imaging.

### Exercise

Visualize the lever-arm distribution for the original full-energy, double-interaction imaging data (i.e. before the position blur was applied).

Do you notice any interesting features in the distribution? If so, what is the reason for these features?

Based on the distribution, choose a lever-arm cutoff. Select the events that have lever-arms greater than this value. N.B. Make sure that your cutoff value isn't so high that it results in you having fewer than `num_cones` events!

Finally, compare the images before and after the lever-arm cut.